In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression #로지스틱 회귀
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier #랜덤포레스트
import tensorflow as tf
from tensorflow.keras import layers
from catboost import Pool,CatBoostClassifier
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score 

In [2]:
tmp_train=pd.read_csv("train.csv").set_index(keys='id')
tmp_test=pd.read_csv("test.csv").set_index(keys='id')
d_code=pd.read_csv("속성_D_코드.csv").set_index(keys='속성 D 코드')
h_code=pd.read_csv("속성_H_코드.csv").set_index(keys='속성 H 코드')
l_code=pd.read_csv("속성_L_코드.csv").set_index(keys='속성 L 코드')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
tmp_train.isna().sum()

In [ ]:
tmp_test.isna().sum()

In [ ]:
def createDF(main_df,d_code=d_code,h_code=h_code,l_code=l_code): #n:세분류,s:소분류,m:중분류,l:대분류
    main_df=main_df.copy()
    main_df["person_prefer_d_1_n"]=main_df["person_prefer_d_1"].apply(lambda x: d_code.loc[x,"속성 D 세분류코드"])
    main_df["person_prefer_d_1_s"]=main_df["person_prefer_d_1"].apply(lambda x: d_code.loc[x,"속성 D 소분류코드"])
    main_df["person_prefer_d_1_m"]=main_df["person_prefer_d_1"].apply(lambda x: d_code.loc[x,"속성 D 중분류코드"])
    main_df["person_prefer_d_1_l"]=main_df["person_prefer_d_1"].apply(lambda x: d_code.loc[x,"속성 D 대분류코드"])
    
    main_df["person_prefer_d_2_n"]=main_df["person_prefer_d_2"].apply(lambda x: d_code.loc[x,"속성 D 세분류코드"])
    main_df["person_prefer_d_2_s"]=main_df["person_prefer_d_2"].apply(lambda x: d_code.loc[x,"속성 D 소분류코드"])
    main_df["person_prefer_d_2_m"]=main_df["person_prefer_d_2"].apply(lambda x: d_code.loc[x,"속성 D 중분류코드"])
    main_df["person_prefer_d_2_l"]=main_df["person_prefer_d_2"].apply(lambda x: d_code.loc[x,"속성 D 대분류코드"])
    
    main_df["person_prefer_d_3_n"]=main_df["person_prefer_d_3"].apply(lambda x: d_code.loc[x,"속성 D 세분류코드"])
    main_df["person_prefer_d_3_s"]=main_df["person_prefer_d_3"].apply(lambda x: d_code.loc[x,"속성 D 소분류코드"])
    main_df["person_prefer_d_3_m"]=main_df["person_prefer_d_3"].apply(lambda x: d_code.loc[x,"속성 D 중분류코드"])
    main_df["person_prefer_d_3_l"]=main_df["person_prefer_d_3"].apply(lambda x: d_code.loc[x,"속성 D 대분류코드"])
    
    main_df['person_prefer_h_1_m']=main_df['person_prefer_h_1'].apply(lambda x: h_code.loc[x,"속성 H 중분류코드"])
    main_df['person_prefer_h_1_l']=main_df['person_prefer_h_1'].apply(lambda x: h_code.loc[x,"속성 H 대분류코드"])
    
    main_df['person_prefer_h_2_m']=main_df['person_prefer_h_2'].apply(lambda x: h_code.loc[x,"속성 H 중분류코드"])
    main_df['person_prefer_h_2_l']=main_df['person_prefer_h_2'].apply(lambda x: h_code.loc[x,"속성 H 대분류코드"])
    
    main_df['person_prefer_h_3_m']=main_df['person_prefer_h_3'].apply(lambda x: h_code.loc[x,"속성 H 중분류코드"])
    main_df['person_prefer_h_3_l']=main_df['person_prefer_h_3'].apply(lambda x: h_code.loc[x,"속성 H 대분류코드"])
    
    main_df['contents_attribute_l_n']=main_df['contents_attribute_l'].apply(lambda x: l_code.loc[x,"속성 L 세분류코드"])
    main_df['contents_attribute_l_s']=main_df['contents_attribute_l'].apply(lambda x: l_code.loc[x,"속성 L 소분류코드"])
    main_df['contents_attribute_l_m']=main_df['contents_attribute_l'].apply(lambda x: l_code.loc[x,"속성 L 중분류코드"])
    main_df['contents_attribute_l_l']=main_df['contents_attribute_l'].apply(lambda x: l_code.loc[x,"속성 L 대분류코드"])
    
    main_df['contents_attribute_d_n']=main_df['contents_attribute_d'].apply(lambda x: d_code.loc[x,"속성 D 세분류코드"])
    main_df['contents_attribute_d_s']=main_df['contents_attribute_d'].apply(lambda x: d_code.loc[x,"속성 D 소분류코드"])
    main_df['contents_attribute_d_m']=main_df['contents_attribute_d'].apply(lambda x: d_code.loc[x,"속성 D 중분류코드"])
    main_df['contents_attribute_d_l']=main_df['contents_attribute_d'].apply(lambda x: d_code.loc[x,"속성 D 대분류코드"])

    main_df['contents_attribute_h_m']=main_df['contents_attribute_h'].apply(lambda x: h_code.loc[x,"속성 H 중분류코드"])
    main_df['contents_attribute_h_l']=main_df['contents_attribute_h'].apply(lambda x: h_code.loc[x,"속성 H 대분류코드"])
    
    return main_df

In [ ]:
def preprocessing(df:pd.DataFrame)->pd.DataFrame:
    print('--------start---------')
    df=df.copy()
    df.drop(['person_prefer_f','person_prefer_g','contents_rn','person_rn'],axis=1,inplace=True)
    print('---------mid_1----------')
    df['d_match_yn']=df['d_s_match_yn'] #d코드 매칭여부
    df['d_n_match_yn']=df['d_m_match_yn'] #d 세분류 매칭여부
    df['d_s_match_yn']=(df['person_prefer_d_1_s']==df['contents_attribute_d_s']) #d 소분류 매칭여부
    df['d_m_matck_yn']=(df['person_prefer_d_1_m']==df['contents_attribute_d_m']) #d 중분류 매칭여부
    
    df['d_2_match_yn']=(df['person_prefer_d_2']==df['contents_attribute_d'])
    df['d_2_n_match_yn']=(df['person_prefer_d_2_n']==df['contents_attribute_d_n'])
    df['d_2_s_match_yn']=(df['person_prefer_d_2_s']==df['contents_attribute_d_s'])
    df['d_2_m_match_yn']=(df['person_prefer_d_2_m']==df['contents_attribute_d_m'])
    df['d_2_l_match_yn']=(df['person_prefer_d_2_l']==df['contents_attribute_d_l'])
    
    df['d_3_match_yn']=(df['person_prefer_d_3']==df['contents_attribute_d'])
    df['d_3_n_match_yn']=(df['person_prefer_d_3_n']==df['contents_attribute_d_n'])
    df['d_3_s_match_yn']=(df['person_prefer_d_3_s']==df['contents_attribute_d_s'])
    df['d_3_m_match_yn']=(df['person_prefer_d_3_m']==df['contents_attribute_d_m'])
    df['d_3_l_match_yn']=(df['person_prefer_d_3_l']==df['contents_attribute_d_l'])
    
    df['h_match_yn']=df['h_s_match_yn']
    
    df['h_2_match_yn']=(df['person_prefer_h_2']==df['contents_attribute_h'])
    df['h_2_m_match_yn']=(df['person_prefer_h_2_m']==df['contents_attribute_h_m'])
    df['h_2_l_match_yn']=(df['person_prefer_h_2_l']==df['contents_attribute_h_l'])
    
    df['h_3_match_yn']=(df['person_prefer_h_3']==df['contents_attribute_h'])
    df['h_3_m_match_yn']=(df['person_prefer_h_3_m']==df['contents_attribute_h_m'])
    df['h_3_l_match_yn']=(df['person_prefer_h_3_l']==df['contents_attribute_h_l'])
    
    #df['a_match_yn']
    df['c_match_yn']=(df['person_prefer_c']==df['contents_attribute_c'])
    df['e_match_yn']=(df['person_prefer_e']==df['contents_attribute_e'])
    
    print('---------mid_2----------')
    for column in df.keys():
        if column=='contents_open_dt':
            df[column]=df[column].astype('datetime64').dt.dayofweek #0~6(0부터 월요일~)
        if df[column].dtypes!='datetime64':
            df[column]=df[column].astype('int64')
    df['weekend']=df['contents_open_dt'].apply(lambda x: 1 if x>=5 else 0)
    print('---------mid_3-----------')
    #df['person_attribute_a']=df['person_attribute_a_1'].apply(lambda x: x+1)*(df['person_attribute_a'].apply(lambda x:1 if x==1 else -1 ))
    #df['contents_attribute_j']=df['contents_attribute_j'].apply(lambda x: 0 if x==1 else 10)+df['contents_attribute_j_1']
    print('---------mid_4----------')
    
    df['contents_attribute_k']-=1
    #df=pd.get_dummies(df,columns=['person_prefer_c','contents_attribute_i','contents_attribute_a',
    #                              'contents_attribute_j','contents_attribute_c','contents_attribute_m','contents_open_dt'])
    df.drop([#'person_attribute_a_1',
             #'contents_attribute_j_1',
             'h_s_match_yn'],axis=1,inplace=True)
    print("-----------finish------------")
    return df

In [ ]:
train_set=preprocessing(createDF(tmp_train))
test_set=preprocessing(createDF(tmp_test))

In [ ]:
train_set.keys()

In [ ]:
test_set.keys()

In [ ]:
key_list=list(train_set.keys())
target_idx=key_list.index('target')
y_train=train_set['target']
x_train=train_set[key_list[:target_idx]+key_list[target_idx+1:]]

In [ ]:
y_train

In [ ]:
x_train

In [ ]:
#로지스틱 회귀
#model=LogisticRegression()
#model.fit(x_train,y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV rf_test=RandomForestClassifier(random_state=25) parameters={'n_estimators':[i for i in range(50,100,5)], 'max_depth':[i for i in range(15,30,3)]} grid_rf=GridSearchCV(rf_test, param_grid = parameters, cv=5, n_jobs=-1 ) grid_rf.fit(x_train,y_train) result=pd.DataFrame(grid_rf.cvresults['params']) result['mean_test_score']=grid_rf.cvresults['mean_test_score'] result.sort_values(by='mean_test_score',ascending=False)

랜덤포레스트
model=RandomForestClassifier(random_state=25,n_estimators=50,max_depth=20) model.fit(x_train,y_train)

인공신경망
model=tf.keras.Sequential()

model.add(layers.Dense(72,input_shape=(72,))) model.add(layers.Activation('relu')) model.add(layers.Dropout(0.2))

model.add(layers.Dense(36)) model.add(layers.Activation('relu')) model.add(layers.Dropout(0.2))

model.add(layers.Dense(1)) model.add(layers.Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])

hist=model.fit(x_train,y_train, epochs=500, batch_size=128, validation_split=0.2, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=250)], verbose=1)



In [ ]:
#캣부스트
#cat_features=x_train.columns[x_train.nunique()>2].to_list()
is_holdout=False
cv=KFold(n_splits=5,
        shuffle=True)
cat_features=x_train.columns[x_train.nunique()>2].to_list()
scores=[]
models=[]
for train_idx,val_idx in cv.split(x_train):
    print('='*60)
    preds=[]
    
    model=CatBoostClassifier(iterations=3000,
                             #task_type='GPU',
                             eval_metric='F1',
                             cat_features=cat_features,
                             one_hot_max_size=4)
    
    model.fit(x_train.iloc[train_idx],y_train[train_idx],
              eval_set=[(x_train.iloc[val_idx],y_train[val_idx])],
              early_stopping_rounds=100,
              verbose=100)
    
    models.append(model)
    scores.append(model.get_best_score()['validation']['F1'])
    if is_holdout:
        break
print(scores)
print(np.mean(scores))

In [ ]:
threshold=0.3425
pred_list=[]
scores_2=[]
for i,(train_idx,val_idx) in enumerate(cv.split(x_train)):
    pred=models[i].predict_proba(x_train.iloc[val_idx])[:,1]
    pred=np.where(pred>=threshold,1,0)
    score=f1_score(y_train[val_idx],pred)
    scores_2.append(score)
    pred=models[i].predict_proba(test_set)[:,1]
    pred_list.append(pred)
print(scores_2)
print(np.mean(scores_2))

In [ ]:
#preds=model.predict(test_set) # catboost 말고
preds=np.mean(pred_list,axis=0) # catboost
preds=np.where(preds>=threshold,1,0) # catboost
submission=pd.read_csv('sample_submission.csv')
submission['target']=preds.round()
submission.to_csv('D:/Users/user/Desktop/submission.csv',index=False)